<h1> This notebook is for Capstone Project</h1>


In [2]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<b> Importing necessary library and web scraping of Wikipedia Page</b> 

In [3]:
import pandas as pd
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


<b> Cleaning data like removing NaN from columns and replacing NaN with logical values.<br></br></b>
<b> Also grouping values based on Postal code and them removing duplicates, if any.</b> 

In [4]:
df1 = df[df['Borough']!='Not assigned']
df1.reset_index(drop=True, inplace=True)
temp = df1.groupby('Postal code')['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()
df_final = df1.merge(temp, on='Postal code').drop(['Neighborhood_y'],axis=1).rename(columns={'Neighborhood_x':'Neighborhood'})
df_final.drop_duplicates(inplace=True)
df_final.shape

(103, 3)

In [5]:
df_final.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
! pip install geocoder

     |████████████████████████████████| 102kB 7.0MB/s ta 0:00:011


<b> Using Geopy Library to extract Latitude and Longitude of the neighborhood of Toronto city</b>

In [7]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()

df_location = pd.DataFrame(columns=['Postal code','Latitude','Longitude'])
for i in range(len(df_final)):
   lo = geolocator.geocode('{},{}, Toronto'.format(df_final.loc[i,'Neighborhood'].split('/')[0].strip(),df_final.loc[i,'Borough']))
   if lo is not None:
     df_location=df_location.append({'Postal code':df_final.loc[i,'Postal code'],'Latitude':lo.latitude,'Longitude':lo.longitude},ignore_index=True) 
   


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


GeocoderUnavailable: Service not available

<b>Geopy libary takes a long time to output the location attributes of the neighborhood</b>


In [8]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df_full = df_final.merge(geo_df, left_on='Postal code', right_on='Postal Code').drop(['Postal Code'],axis=1)
df_full.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [10]:
geolocator = Nominatim(user_agent="to_myself")
location = geolocator.geocode('Toronto, Canada')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.6534817, -79.3839347.


In [12]:
!pip install folium

     |████████████████████████████████| 92kB 6.9MB/s eta 0:00:011


In [13]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(df_full['Latitude'], df_full['Longitude'], df_full['Borough'], df_full['Neighborhood']):
    label = 'Neighborhood: {}, Borough: {}'.format(neighborhood, borough)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto    